In [1]:
import yfinance as yf
import bs4 as bs
import pickle
import requests
import pandas as pd
import json
from dateutil import parser
from datetime import datetime
import os
import time
from pandas.io.json import json_normalize
from alpha_vantage.timeseries import TimeSeries
from alpha_vantage.fundamentaldata import FundamentalData
import wrds
from datetime import datetime as dt
import numpy as np
import glob, os
import csv
from matplotlib import pyplot as plt
pd.set_option("display.precision", 8)

import os.path
from os import path




%load_ext autoreload
%autoreload 2

In [2]:
start_download = '2006-06-01'
end_download = '2021-06-01'

In [3]:
start_date = '2009-06-30' # Define start and end dates
end_date = '2020-04-15'

### Gets currently listed SP500 companies and thier CIKs

In [4]:
# gets list of SP500 companies and their CIKs

#standard: pull from yahoo finance, yfinance
#

def save_SP500_tickers():
    resp = requests.get('https://web.archive.org/web/20200411084847/https://en.wikipedia.org/wiki/List_of_S%26P_500_companies')
    soup = bs.BeautifulSoup(resp.text, 'lxml')
    table = soup.find('table', {'class': 'wikitable sortable'})
    tickers = []
    ciks = []
    
    for row in table.findAll('tr')[1:]:
        ticker = row.findAll('td')[0].text[:-1]
        cik = row.findAll('td')[7].text[:]
        
        tickers.append(ticker)
        ciks.append(cik)
    
    #drops '\n' from certain entries in CIK list
    ciks = list(map(lambda x:x.strip(),ciks))
    
    tickers_ciks_df = pd.DataFrame({'ticker': tickers, 'cik': ciks})

        
    return tickers_ciks_df
    

### Prepping tickers list

## Remove tickers from list if change detected in change list

In [5]:
changes = pd.read_csv('S&P 500 Historical Components & Changes(10-14-2022).csv',
                              header=0,
                              index_col=0,
                              parse_dates=True)

C:\Users\Daniel\AppData\Local\Temp\ipykernel_10788\2208461315.py:1: DtypeWarning: Columns (500,501,502,503,504,505,506,507) have mixed types. Specify dtype option on import or set low_memory=False.
  changes = pd.read_csv('S&P 500 Historical Components & Changes(10-14-2022).csv',


In [6]:
tickers = sorted(set.intersection(*list(changes[start_date:end_date].apply(lambda x: set(x.dropna()), axis=1))))

## Returns

> ### Download Returns Data from Yfinance

In [7]:
  
if not path.exists("Export\OHLC_data"):
    data_yf = yf.download(tickers, start=start_download, end=end_download)
    data_yf.index = pd.to_datetime(data_yf.index, format = '%Y-%m-%d')
    with open('Export/OHLC_data', 'wb') as fh:
        pickle.dump(data_yf, fh)
    
    print('\n dumped \'data_yf\' to pickle as OHLC')
    
elif path.exists("Export\OHLC_data"):
    # load from pickle
    with open("Export\OHLC_data","rb") as fin:
        data_yf = pickle.load(fin)
    data_yf.index = pd.to_datetime(data_yf.index, format = '%Y-%m-%d')
    print('loaded OHLC data from pickle as \'data_yf\'')


loaded OHLC data from pickle as 'data_yf'


In [8]:
data_yf.index.dtype

dtype('<M8[ns]')

> ### Calculate Daily Returns: Training Period and Covid(testing) Period (PERCENT CHANGE)

In [9]:
#rets = (data_yf['Open'] - data_yf['Close'])/data_yf['Open']
rets_PC = data_yf['Adj Close'].pct_change()  

In [10]:
rets_PC = rets_PC.dropna(how='all',axis=0)

In [11]:
rets_PC = rets_PC.dropna(axis=1)

In [12]:
returns_PC = rets_PC[pd.to_datetime('2009-06-30', format = '%Y-%m-%d'): pd.to_datetime('2020-02-03', format = '%Y-%m-%d')]

In [13]:
returns_covid_PC = rets_PC[pd.to_datetime('2020-02-03', format = '%Y-%m-%d'):pd.to_datetime('2020-04-14', format = '%Y-%m-%d')]

> ### Calculate Daily Returns: Training Period and Covid(testing) Period (OCO)

In [14]:
rets_OCO = (data_yf['Close']-data_yf['Open'])/(data_yf['Open'])

In [15]:
rets_OCO = rets_OCO.dropna(how='all',axis=0)

In [16]:
rets_OCO = rets_OCO.dropna(axis=1)

In [17]:
returns_OCO = rets_OCO[pd.to_datetime('2009-06-30', format = '%Y-%m-%d'): pd.to_datetime('2020-02-03', format = '%Y-%m-%d')]

In [18]:
returns_covid_OCO = rets_OCO[pd.to_datetime('2020-02-03', format = '%Y-%m-%d'):pd.to_datetime('2020-04-14', format = '%Y-%m-%d')]

## Export Returns during Covid and Testing Periods

In [19]:
#Percent Change Version
returns_PC.T.to_csv('Export/returns_PC.csv')
returns_covid_PC.T.to_csv('Export/returns_covid_PC.csv')

In [20]:
#((Open - Close)/Open) Version
returns_OCO.T.to_csv('Export/returns_OCO.csv')
returns_covid_OCO.T.to_csv('Export/returns_covid_OCO.csv')

> ### Calculate Monthly Returns: Training Period and Testing Period

In [21]:
#monthly returns fr % change method
monthly_returns_PC = (returns_PC+1).groupby(pd.Grouper(freq="M")).prod() -1
monthly_returns_covid_PC = (returns_covid_PC+1).groupby(pd.Grouper(freq="M")).prod() -1

In [22]:
#monthly returns for OCO method
monthly_returns_OCO = (returns_OCO+1).groupby(pd.Grouper(freq="M")).prod() -1
monthly_returns_covid_OCO = (returns_covid_OCO+1).groupby(pd.Grouper(freq="M")).prod() -1

## T-Bills

In [23]:
t_test = pd.read_csv("DTB3.csv",
                   header=0, index_col=0, parse_dates=True, na_values=-99.99)


In [24]:
t_test = t_test.replace('.', np.nan).astype(float).ffill()/100

In [25]:
t_test['2009-06-30':'2020-02-03']

,3MO TBILL
DATE,
2009-06-30,0.0019
2009-07-01,0.0017
2009-07-02,0.0017
2009-07-03,0.0017
2009-07-06,0.0019
...,...
2020-01-28,0.0154
2020-01-29,0.0153
2020-01-30,0.0154


In [26]:
t_test = t_test/63

In [27]:
merged_test = returns_PC.merge(t_test, how = 'left', left_index=True, right_index=True)
#merged_diff = returns.merge(tbills_diff, how = 'left', left_index=True, right_index=True)

In [28]:
returns = merged_test.iloc[:,:-1].ffill().bfill()

In [29]:
t_test

,3MO TBILL
DATE,
2009-06-02,0.00002222
2009-06-03,0.00002222
2009-06-04,0.00002222
2009-06-05,0.00003016
2009-06-08,0.00003016
...,...
2020-10-12,0.00001587
2020-10-13,0.00001746
2020-10-14,0.00001905


In [30]:
t_test = pd.DataFrame(merged_test['3MO TBILL'])

In [31]:
t_test

,3MO TBILL
Date,
2009-06-30,0.00003016
2009-07-01,0.00002698
2009-07-02,0.00002698
2009-07-06,0.00003016
2009-07-07,0.00003016
...,...
2020-01-28,0.00024444
2020-01-29,0.00024286
2020-01-30,0.00024444


In [32]:
0.00024444*63

0.01539972

In [33]:
t_test = t_test.replace('.', np.nan).astype(float).ffill()/100

In [34]:
t_test['2009-06-30':'2020-02-03']

,3MO TBILL
Date,
2009-06-30,0.00000030
2009-07-01,0.00000027
2009-07-02,0.00000027
2009-07-06,0.00000030
2009-07-07,0.00000030
...,...
2020-01-28,0.00000244
2020-01-29,0.00000243
2020-01-30,0.00000244


> ### T-Bills Training

In [35]:
# read csv from file and set column name
t = pd.read_csv("DTB3.csv",
                   header=0, index_col=0, parse_dates=True, na_values=-99.99)
t.columns = ['tbill']

In [36]:
# convert from % return to decimal return
tbills = t.replace('.', np.nan).astype(float).ffill()/100
#sets date to training period
tbills = tbills/63
tbills = tbills['2009-06-30':'2020-02-03']


In [37]:
#convert from % return to decimal return
tbills_diff =  t.replace('.', np.nan).astype(float).ffill()/100
#sets date to training period, takes difference between each days rates and fills NaN values
tbills_diff=tbills_diff/63
tbills_diff = tbills_diff['2009-06-30':'2020-02-03'].diff().bfill()


In [38]:
tbills

,tbill
DATE,
2009-06-30,0.00003016
2009-07-01,0.00002698
2009-07-02,0.00002698
2009-07-03,0.00002698
2009-07-06,0.00003016
...,...
2020-01-28,0.00024444
2020-01-29,0.00024286
2020-01-30,0.00024444


In [39]:
tbills_diff

,tbill
DATE,
2009-06-30,-0.00000317
2009-07-01,-0.00000317
2009-07-02,0.00000000
2009-07-03,0.00000000
2009-07-06,0.00000317
...,...
2020-01-28,0.00000317
2020-01-29,-0.00000159
2020-01-30,0.00000159


> ### T-Bills Covid

In [40]:
# read csv from file and set column name
t = pd.read_csv("DTB3.csv",
                   header=0, index_col=0, parse_dates=True, na_values=-99.99)
t.columns = ['tbill']

In [41]:
# convert from % return to decimal return
tbills_covid = t.replace('.', np.nan).astype(float).ffill()/100
#sets date to training period
tbills_covid = tbills_covid/63
tbills_covid = tbills_covid['2020-02-03':'2020-04-14']

In [42]:
#convert from % return to decimal return
tbills_covid_diff = t.replace('.', np.nan).astype(float).ffill()/100
tbills_covid_diff = tbills_covid_diff/63
#sets date to training period, takes difference between each days rates and fills NaN values
tbills_covid_diff = tbills_covid_diff['2020-02-03':'2020-04-14'].diff().bfill()

In [43]:
tbills_covid

,tbill
DATE,
2020-02-03,0.00024444
2020-02-04,0.00024444
2020-02-05,0.00024444
2020-02-06,0.00024444
2020-02-07,0.00024286
2020-02-10,0.00024603
2020-02-11,0.00024444
2020-02-12,0.00024603
2020-02-13,0.00024762


In [44]:
tbills_covid_diff

,tbill
DATE,
2020-02-03,0.00000000
2020-02-04,0.00000000
2020-02-05,0.00000000
2020-02-06,0.00000000
2020-02-07,-0.00000159
2020-02-10,0.00000317
2020-02-11,-0.00000159
2020-02-12,0.00000159
2020-02-13,0.00000159


In [45]:
tbills_covid.shape

(52, 1)

> ### Training Period: Match Returns and T-Bills Index

In [46]:
merged = returns.merge(tbills, how = 'left', left_index=True, right_index=True)
merged_diff = returns.merge(tbills_diff, how = 'left', left_index=True, right_index=True)

In [47]:
merged

,A,AAPL,ABC,ABT,ADBE,ADI,ADM,ADP,ADSK,AEE,...,WY,WYNN,XEL,XOM,XRAY,XRX,YUM,ZBH,ZION,tbill
Date,,,,,,,,,,,,,,,,,,,,,
2009-06-30,-0.00587349,0.00324017,-0.01826198,-0.01631134,-0.01118105,0.00405194,-0.01761489,0.00425018,-0.02865923,-0.00200544,...,-0.00555527,-0.04413755,-0.00162688,-0.00949279,-0.01035911,-0.02114771,0.00059997,-0.00907233,-0.05555517,0.00003016
2009-07-01,0.01526303,0.00280841,0.02818532,-0.00850357,0.01272087,0.00322817,0.02913713,-0.00253974,-0.01685984,0.02008842,...,0.01117300,0.00254950,0.02009799,0.00929765,-0.00294432,0.02006137,0.05038968,-0.01713564,-0.01816628,0.00002698
2009-07-02,-0.04364674,-0.01967347,-0.02302657,-0.00771879,-0.03558969,-0.00643567,-0.03666050,-0.03309734,-0.02947477,-0.04568724,...,-0.04094889,-0.05594780,-0.02502728,-0.02933662,-0.02099698,-0.03177030,-0.02198723,-0.03940807,-0.02555057,0.00002698
2009-07-06,-0.00862072,-0.01007011,-0.00168390,-0.00604984,-0.01736612,-0.02510120,0.00791229,0.00263301,0.00331306,0.00577817,...,0.01626568,-0.02155073,0.01310828,-0.00569425,-0.01306964,-0.01874971,0.01751812,0.00124264,0.00632925,0.00003016
2009-07-07,-0.02864450,-0.02315847,0.02304660,-0.01434784,-0.03019145,-0.01162802,0.01794445,-0.02742921,-0.02641715,-0.02995492,...,-0.07102387,-0.05934542,-0.02048540,-0.02261391,-0.02376930,-0.03662417,-0.01004254,-0.01291228,-0.00988313,0.00003016
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2020-01-28,0.00716278,0.02828946,0.01816712,-0.00055807,0.02228311,0.01871470,0.00895909,0.01456689,0.02123860,0.00262164,...,0.00729188,0.00871742,0.00448309,-0.00139024,-0.01088231,0.04935794,0.00411908,0.00783594,0.01356009,0.00024444
2020-01-29,0.00068816,0.02093218,0.00841671,-0.00446729,-0.00837493,-0.00944774,-0.00546421,-0.04071785,-0.00144975,0.00871486,...,-0.00954262,-0.00896221,0.00297498,-0.00835262,-0.00253892,-0.00135962,0.01726767,0.01038870,-0.01528974,0.00024286
2020-01-30,-0.03278328,-0.00144891,-0.01090581,0.00022453,0.01444573,-0.00759526,0.04853461,0.02321094,-0.00375488,0.01086145,...,-0.00099675,0.02236582,0.01987558,0.01060683,-0.02206008,0.00680650,-0.00028128,0.00285240,0.00841051,0.00024444


In [48]:
returns = merged.iloc[:,:-1].ffill().bfill()

In [49]:
tbills = pd.DataFrame(merged['tbill'])
tbills_diff = pd.DataFrame(merged_diff['tbill'])

In [50]:
returns

,A,AAPL,ABC,ABT,ADBE,ADI,ADM,ADP,ADSK,AEE,...,WMT,WY,WYNN,XEL,XOM,XRAY,XRX,YUM,ZBH,ZION
Date,,,,,,,,,,,,,,,,,,,,,
2009-06-30,-0.00587349,0.00324017,-0.01826198,-0.01631134,-0.01118105,0.00405194,-0.01761489,0.00425018,-0.02865923,-0.00200544,...,-0.00656314,-0.00555527,-0.04413755,-0.00162688,-0.00949279,-0.01035911,-0.02114771,0.00059997,-0.00907233,-0.05555517
2009-07-01,0.01526303,0.00280841,0.02818532,-0.00850357,0.01272087,0.00322817,0.02913713,-0.00253974,-0.01685984,0.02008842,...,-0.00144531,0.01117300,0.00254950,0.02009799,0.00929765,-0.00294432,0.02006137,0.05038968,-0.01713564,-0.01816628
2009-07-02,-0.04364674,-0.01967347,-0.02302657,-0.00771879,-0.03558969,-0.00643567,-0.03666050,-0.03309734,-0.02947477,-0.04568724,...,-0.01199035,-0.04094889,-0.05594780,-0.02502728,-0.02933662,-0.02099698,-0.03177030,-0.02198723,-0.03940807,-0.02555057
2009-07-06,-0.00862072,-0.01007011,-0.00168390,-0.00604984,-0.01736612,-0.02510120,0.00791229,0.00263301,0.00331306,0.00577817,...,-0.00125565,0.01626568,-0.02155073,0.01310828,-0.00569425,-0.01306964,-0.01874971,0.01751812,0.00124264,0.00632925
2009-07-07,-0.02864450,-0.02315847,0.02304660,-0.01434784,-0.03019145,-0.01162802,0.01794445,-0.02742921,-0.02641715,-0.02995492,...,0.00230446,-0.07102387,-0.05934542,-0.02048540,-0.02261391,-0.02376930,-0.03662417,-0.01004254,-0.01291228,-0.00988313
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2020-01-28,0.00716278,0.02828946,0.01816712,-0.00055807,0.02228311,0.01871470,0.00895909,0.01456689,0.02123860,0.00262164,...,0.00638691,0.00729188,0.00871742,0.00448309,-0.00139024,-0.01088231,0.04935794,0.00411908,0.00783594,0.01356009
2020-01-29,0.00068816,0.02093218,0.00841671,-0.00446729,-0.00837493,-0.00944774,-0.00546421,-0.04071785,-0.00144975,0.00871486,...,-0.00608904,-0.00954262,-0.00896221,0.00297498,-0.00835262,-0.00253892,-0.00135962,0.01726767,0.01038870,-0.01528974
2020-01-30,-0.03278328,-0.00144891,-0.01090581,0.00022453,0.01444573,-0.00759526,0.04853461,0.02321094,-0.00375488,0.01086145,...,0.00595384,-0.00099675,0.02236582,0.01987558,0.01060683,-0.02206008,0.00680650,-0.00028128,0.00285240,0.00841051


In [51]:
tbills

,tbill
Date,
2009-06-30,0.00003016
2009-07-01,0.00002698
2009-07-02,0.00002698
2009-07-06,0.00003016
2009-07-07,0.00003016
...,...
2020-01-28,0.00024444
2020-01-29,0.00024286
2020-01-30,0.00024444


In [52]:
tbills_diff

,tbill
Date,
2009-06-30,-0.00000317
2009-07-01,-0.00000317
2009-07-02,0.00000000
2009-07-06,0.00000317
2009-07-07,0.00000000
...,...
2020-01-28,0.00000317
2020-01-29,-0.00000159
2020-01-30,0.00000159


> ### Testing Period: Match Returns and T-Bills Index

In [53]:
merged_covid = returns_covid_PC.merge(tbills_covid, how = 'left', left_index=True, right_index=True)
merged_covid_diff = returns_covid_PC.merge(tbills_covid_diff, how = 'left', left_index=True, right_index=True)

In [54]:
returns_covid = merged_covid.iloc[:,:-1].ffill().bfill()

In [55]:
tbills_covid = pd.DataFrame(merged_covid['tbill']).ffill().bfill()

In [56]:
tbills_covid_diff = pd.DataFrame(merged_covid_diff['tbill']).ffill().bfill()

In [57]:
returns_covid.shape

(50, 289)

In [58]:
tbills_covid.shape

(50, 1)

In [59]:
tbills_covid_diff.shape

(50, 1)

> ### Training Period and Testing Period: Monthly T-Bills

In [60]:
tbills_monthly = (tbills+1).groupby(pd.Grouper(freq="M")).prod() -1
tbills_diff_monthly = (tbills_diff).groupby(pd.Grouper(freq="M")).prod()

tbills_covid_monthly = (tbills_covid+1).groupby(pd.Grouper(freq="M")).prod().ffill() -1
tbills_monthly_covid_diff = (tbills_covid_diff).groupby(pd.Grouper(freq="M")).mean()

## Calculate Sharpe Ratios

> ### Sharpes calcs with Excess - Direct (PC)

In [61]:
excess_returns_PC = returns_PC.sub(tbills['tbill'], axis = 0)
excess_returns_monthly_PC = monthly_returns_PC.sub(tbills_monthly['tbill'], axis = 0)

In [62]:
sharpes_PC = pd.DataFrame(excess_returns_PC.mean()/returns_PC.std())
#sharpes = pd.DataFrame(returns.mean()/returns.std())


sharpes_monthly_PC = pd.DataFrame(excess_returns_monthly_PC.mean()/monthly_returns_PC.std())
#sharpes_monthly = pd.DataFrame(monthly_returns.mean()/monthly_returns.std())

sharpes_PC = sharpes_PC*(252)**0.5
sharpes_monthly_PC = sharpes_monthly_PC*(12)**0.5

In [63]:
sharpes_PC.columns = ['sharpes']
sharpes_monthly_PC.columns = ['sharpes']

In [64]:
sharpes_PC.index.name = 'ticker'
sharpes_monthly_PC.index.name = 'ticker'

In [65]:
sharpes_PC.sort_values(by='sharpes')

,sharpes
ticker,
DVN,-0.09174980
APA,-0.08042710
SLB,-0.02023952
OXY,0.01743242
NOV,0.05621424
...,...
CMS,1.13490714
FISV,1.14623849
MA,1.16426396


In [66]:
sharpes_monthly_PC.sort_values(by='sharpes')

,sharpes
ticker,
APA,-0.08063527
DVN,-0.06774487
SLB,-0.00564712
OXY,0.00844826
NOV,0.04341347
...,...
CMS,1.26435777
NI,1.28138520
FISV,1.35116017


In [67]:
sharpes_PC.sort_values(by='sharpes').mean()

sharpes    0.62149321
dtype: float64

In [68]:
sharpes_monthly_PC.sort_values(by='sharpes').mean()

sharpes    0.66019716
dtype: float64

### EXPORT

In [69]:
sharpes_PC.to_csv('Export/sharpes_PC.csv')
sharpes_monthly_PC.to_csv('Export/sharpes_monthly_PC.csv')

> ### Sharpes calcs with Excess - Direct (OCO)

In [70]:
excess_returns_OCO = returns_OCO.sub(tbills['tbill'], axis = 0)
excess_returns_monthly_OCO = monthly_returns_OCO.sub(tbills_monthly['tbill'], axis = 0)

In [71]:
sharpes_OCO = pd.DataFrame(excess_returns_OCO.mean()/returns_OCO.std())
#sharpes = pd.DataFrame(returns.mean()/returns.std())


sharpes_monthly_OCO = pd.DataFrame(excess_returns_monthly_OCO.mean()/monthly_returns_OCO.std())
#sharpes_monthly = pd.DataFrame(monthly_returns.mean()/monthly_returns.std())

sharpes_OCO = sharpes_OCO*(252)**0.5
sharpes_monthly_OCO = sharpes_monthly_OCO*(12)**0.5

In [72]:
sharpes_OCO.columns = ['sharpes']
sharpes_monthly_OCO.columns = ['sharpes']

In [73]:
sharpes_OCO.index.name = 'ticker'
sharpes_monthly_OCO.index.name = 'ticker'

In [74]:
sharpes_OCO.sort_values(by='sharpes')

,sharpes
ticker,
F,-0.84279601
BAC,-0.72981199
FCX,-0.52835816
C,-0.51414508
DVN,-0.49781592
...,...
AON,1.20757081
HSY,1.23984924
SHW,1.31123173


In [75]:
sharpes_monthly_OCO.sort_values(by='sharpes')

,sharpes
ticker,
F,-0.79212502
BAC,-0.61764227
MU,-0.49637062
MRO,-0.46358456
DVN,-0.46166430
...,...
MMC,1.13392206
HSY,1.15673264
SHW,1.29382253


In [76]:
sharpes_OCO.sort_values(by='sharpes').mean()

sharpes    0.37960045
dtype: float64

In [77]:
sharpes_monthly_OCO.sort_values(by='sharpes').mean()

sharpes    0.38424593
dtype: float64

### EXPORT

In [78]:
sharpes_OCO.to_csv('Export/sharpes_OCO.csv')
sharpes_monthly_OCO.to_csv('Export/sharpes_monthly_OCO.csv')

> ### Sharpes calcs with Excess - Diff (PC)

In [79]:
excess_returns_diff_PC = returns_PC.sub(tbills_diff['tbill'], axis = 0)
excess_returns_monthly_diff_PC = monthly_returns_PC.sub(tbills_diff_monthly['tbill'], axis = 0)

In [80]:
sharpes_diff_PC = pd.DataFrame(excess_returns_diff_PC.mean()/returns_PC.std())
sharpes_monthly_diff_PC = pd.DataFrame(excess_returns_monthly_diff_PC.mean()/returns_PC.std())

sharpes_diff_PC = sharpes_diff_PC*(252)**0.5
sharpes_monthly_diff_PC = sharpes_monthly_diff_PC*(12)**0.5

In [81]:
sharpes_diff_PC.columns = ['sharpes']
sharpes_monthly_diff_PC.columns = ['sharpes']

In [82]:
sharpes_diff_PC.index.name = 'ticker'
sharpes_monthly_diff_PC.index.name = 'ticker'

In [83]:
sharpes_diff_PC.sort_values(by='sharpes')

,sharpes
ticker,
DVN,-0.03152377
APA,-0.02079241
SLB,0.05789284
OXY,0.10078582
NOV,0.11964186
...,...
MA,1.25146689
FISV,1.26122920
CMS,1.26710887


In [84]:
sharpes_monthly_diff_PC.sort_values(by='sharpes')

,sharpes
ticker,
APA,-0.09392857
DVN,-0.05019335
SLB,0.32692007
OXY,0.41476812
NOV,0.47844740
...,...
WEC,5.49753594
FISV,5.58856041
CMS,5.66944295


In [85]:
sharpes_diff_PC.sort_values(by='sharpes').mean()

sharpes    0.71476901
dtype: float64

In [86]:
sharpes_monthly_diff_PC.sort_values(by='sharpes').mean()

sharpes    3.16544569
dtype: float64

### EXPORT

In [87]:
sharpes_diff_PC.to_csv('Export/sharpes_diff_PC.csv')
sharpes_monthly_diff_PC.to_csv('Export/sharpes_monthly_diff_PC.csv')

> ### Sharpes calcs with Excess - Diff (OCO)

In [88]:
excess_returns_diff_OCO = returns_OCO.sub(tbills_diff['tbill'], axis = 0)
excess_returns_monthly_diff_OCO = monthly_returns_OCO.sub(tbills_diff_monthly['tbill'], axis = 0)

In [89]:
sharpes_diff_OCO = pd.DataFrame(excess_returns_diff_OCO.mean()/returns_OCO.std())
sharpes_monthly_diff_OCO = pd.DataFrame(excess_returns_monthly_diff_OCO.mean()/returns_OCO.std())

sharpes_diff_OCO = sharpes_diff_OCO*(252)**0.5
sharpes_monthly_diff_OCO = sharpes_monthly_diff_OCO*(12)**0.5

In [90]:
sharpes_diff_OCO.columns = ['sharpes']
sharpes_monthly_diff_OCO.columns = ['sharpes']

In [91]:
sharpes_diff_OCO.index.name = 'ticker'
sharpes_monthly_diff_OCO.index.name = 'ticker'

In [92]:
sharpes_diff_OCO.sort_values(by='sharpes')

,sharpes
ticker,
F,-0.74986674
BAC,-0.64687686
FCX,-0.47297146
C,-0.42991245
DVN,-0.42561943
...,...
AON,1.34270966
HSY,1.37910655
SHW,1.42862237


In [93]:
sharpes_monthly_diff_OCO.sort_values(by='sharpes')

,sharpes
ticker,
F,-3.30387321
BAC,-2.72250945
DVN,-1.86127420
C,-1.78746870
MRO,-1.74676509
...,...
AON,6.16439590
HSY,6.31312466
SHW,6.50743824


In [94]:
sharpes_diff_OCO.sort_values(by='sharpes').mean()

sharpes    0.49054685
dtype: float64

In [95]:
sharpes_monthly_diff_OCO.sort_values(by='sharpes').mean()

sharpes    2.21108448
dtype: float64

### EXPORT

In [96]:
sharpes_diff_OCO.to_csv('Export/sharpes_diff_OCO.csv')
sharpes_monthly_diff_OCO.to_csv('Export/sharpes_monthly_diff_OCO.csv')

> ### Sharpes, No Excess (PC)

In [97]:
sharpes_no_excess_PC = pd.DataFrame(returns_PC.mean()/returns_PC.std())
sharpes_monthly_no_excess_PC = pd.DataFrame(monthly_returns_PC.mean()/monthly_returns_PC.std())

sharpes_no_excess_PC = sharpes_no_excess_PC*(252)**0.5
sharpes_monthly_no_excess_PC = sharpes_monthly_no_excess_PC*(12)**0.5

In [98]:
sharpes_no_excess_PC.columns = ['sharpes']
sharpes_monthly_no_excess_PC.columns = ['sharpes']

In [99]:
sharpes_no_excess_PC.index.name = 'ticker'
sharpes_monthly_no_excess_PC.index.name = 'ticker'

In [100]:
sharpes_no_excess_PC.sort_values(by='sharpes')

,sharpes
ticker,
DVN,-0.03146977
APA,-0.02073894
SLB,0.05796290
OXY,0.10086056
NOV,0.11969874
...,...
MA,1.25154508
FISV,1.26133231
CMS,1.26722742


In [101]:
sharpes_monthly_no_excess_PC.sort_values(by='sharpes')

,sharpes
ticker,
APA,-0.02081296
DVN,-0.01053234
SLB,0.06888771
OXY,0.09366193
NOV,0.10857913
...,...
CMS,1.41358454
NI,1.41995511
FISV,1.48995177


In [102]:
sharpes_no_excess_PC.sort_values(by='sharpes').mean()

sharpes    0.71485265
dtype: float64

In [103]:
sharpes_monthly_no_excess_PC.sort_values(by='sharpes').mean()

sharpes    0.76145786
dtype: float64

### EXPORT

In [104]:
sharpes_no_excess_PC.to_csv('Export/sharpes_no_excess_PC.csv')
sharpes_monthly_no_excess_PC.to_csv('Export/sharpes_monthly_no_excess_PC.csv')

> ### Sharpes, No Excess (OCO)

In [105]:
sharpes_no_excess_OCO = pd.DataFrame(returns_OCO.mean()/returns_OCO.std())
sharpes_monthly_no_excess_OCO = pd.DataFrame(monthly_returns_OCO.mean()/monthly_returns_OCO.std())

sharpes_no_excess_OCO = sharpes_no_excess_OCO*(252)**0.5
sharpes_monthly_no_excess_OCO = sharpes_monthly_no_excess_OCO*(12)**0.5

In [106]:
sharpes_no_excess_OCO.columns = ['sharpes']
sharpes_monthly_no_excess_OCO.columns = ['sharpes']

In [107]:
sharpes_no_excess_OCO.index.name = 'ticker'
sharpes_monthly_no_excess_OCO.index.name = 'ticker'

In [108]:
sharpes_no_excess_OCO.sort_values(by='sharpes')

,sharpes
ticker,
F,-0.74978341
BAC,-0.64680250
FCX,-0.47292179
C,-0.42983692
DVN,-0.42555469
...,...
AON,1.34283084
HSY,1.37923142
SHW,1.42872764


In [109]:
sharpes_monthly_no_excess_OCO.sort_values(by='sharpes')

,sharpes
ticker,
F,-0.70265099
BAC,-0.54278634
MU,-0.43124417
MRO,-0.39511803
DVN,-0.39270399
...,...
MMC,1.27804366
HSY,1.28506196
SHW,1.40888311


In [110]:
sharpes_no_excess_OCO.sort_values(by='sharpes').mean()

sharpes    0.49064633
dtype: float64

In [111]:
sharpes_monthly_no_excess_OCO.sort_values(by='sharpes').mean()

sharpes    0.49659568
dtype: float64

### EXPORT

In [112]:
sharpes_no_excess_OCO.to_csv('Export/sharpes_no_excess_OCO.csv')
sharpes_monthly_no_excess_OCO.to_csv('Export/sharpes_monthly_no_excess_OCO.csv')

## Export Data

> ### Returns, Covid Returns, Monthly Sharpes (using difference method, 10 YR T-Bills)

> ### Returns, Covid Returns, Daily Sharpes (using difference method, 10 YR T-Bills)

> ### Returns, Covid Returns, Monthly Sharpes (Direct method, 10 YR T-Bills)

> ### Returns, Covid Returns, Daily Sharpes (Direct method, 10 YR T-Bills)